<a href="https://colab.research.google.com/github/sanhiitaa/100-days-of-deep-learning/blob/main/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importing libraries and loading dataset via API command**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# making the root directory to store api token
!mkdir -p /root/.kaggle

# moving the api token into the root directory
!mv kaggle.json /root/.kaggle

# importing kaggle
import kaggle

In [3]:
# downloading the dataset via api
!kaggle datasets download -d uciml/pima-indians-diabetes-database

  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 21.0MB/s]


In [4]:
# unzipping the file
!unzip pima-indians-diabetes-database


Archive:  pima-indians-diabetes-database.zip
  inflating: diabetes.csv            


In [5]:
# loading dataset into the dataframe
df=pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [7]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [8]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [9]:
from sklearn.preprocessing import StandardScaler

scale=StandardScaler()
X=scale.fit_transform(X)

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [52]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [12]:
model= Sequential()

model.add(Dense(64, activation='relu', input_dim=8))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2689 (10.50 KB)
Trainable params: 2689 (10.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# compiling the model
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# fitting the model
model.fit(x_train, y_train, batch_size=32, epochs=150, validation_data=(x_test, y_test))

Epoch 1/150
20/20 [==============================] - 1s 15ms/step - loss: 0.6542 - accuracy: 0.6270 - val_loss: 0.5994 - val_accuracy: 0.7338
Epoch 2/150
20/20 [==============================] - 0s 5ms/step - loss: 0.5545 - accuracy: 0.7362 - val_loss: 0.5317 - val_accuracy: 0.7792
Epoch 3/150
20/20 [==============================] - 0s 4ms/step - loss: 0.5022 - accuracy: 0.7410 - val_loss: 0.4988 - val_accuracy: 0.7792
Epoch 4/150
20/20 [==============================] - 0s 4ms/step - loss: 0.4768 - accuracy: 0.7671 - val_loss: 0.4822 - val_accuracy: 0.7922
Epoch 5/150
20/20 [==============================] - 0s 4ms/step - loss: 0.4628 - accuracy: 0.7736 - val_loss: 0.4786 - val_accuracy: 0.7922
Epoch 6/150
20/20 [==============================] - 0s 4ms/step - loss: 0.4524 - accuracy: 0.7818 - val_loss: 0.4747 - val_accuracy: 0.7922
Epoch 7/150
20/20 [==============================] - 0s 4ms/step - loss: 0.4465 - accuracy: 0.7785 - val_loss: 0.4723 - val_accuracy: 0.7857
Epoch 8/150


### **Installing and importing keras_tuner**

In [15]:
# installing keras-tuner
!pip install -q keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.8 MB/s eta 0:00:00


In [16]:
# importing keras_tuner
import keras_tuner as kt

### **Selecting optimizer**

In [17]:
def build_model(hyperpara):
  # define mode
  model= Sequential()

  #add layers
  model.add(Dense(64, activation='relu', input_dim=8))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # setting up optimiers to be tested
  optimizer= hyperpara.Choice('optimizer', values= ['adam', 'sgd', 'rmsprop', 'adadelta'])

  #compile model
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [18]:
# setting up a tuner object
tuner= kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

In [19]:
# searching for the best parameter
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.5454545617103577

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 09s


In [20]:
# extracting the best parameter
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [21]:
# extracting the best model
best_model= tuner.get_best_models(num_models=1)[0]

In [22]:
# best model summary
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2689 (10.50 KB)
Trainable params: 2689 (10.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# training the best model
best_model.fit(x_train, y_train, batch_size=32, epochs=150, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/150
20/20 [==============================] - 1s 13ms/step - loss: 0.5012 - accuracy: 0.7590 - val_loss: 0.4754 - val_accuracy: 0.7597
Epoch 8/150
20/20 [==============================] - 0s 4ms/step - loss: 0.4776 - accuracy: 0.7655 - val_loss: 0.4579 - val_accuracy: 0.7662
Epoch 9/150
20/20 [==============================] - 0s 3ms/step - loss: 0.4623 - accuracy: 0.7801 - val_loss: 0.4522 - val_accuracy: 0.7727
Epoch 10/150
20/20 [==============================] - 0s 4ms/step - loss: 0.4545 - accuracy: 0.7915 - val_loss: 0.4504 - val_accuracy: 0.7727
Epoch 11/150
20/20 [==============================] - 0s 4ms/step - loss: 0.4505 - accuracy: 0.7850 - val_loss: 0.4467 - val_accuracy: 0.7792
Epoch 12/150
20/20 [==============================] - 0s 4ms/step - loss: 0.4453 - accuracy: 0.7883 - val_loss: 0.4539 - val_accuracy: 0.7662
Epoch 13/150
20/20 [==============================] - 0s 4ms/step - loss: 0.4420 - accuracy: 0.7899 - val_loss: 0.4496 - val_accuracy: 0.7792
Epoch 14

### **Selecting number of nodes**

In [24]:
def model_builder(hp):
  # creating Sequrntial object
  model =Sequential()

  # defining number of neuron units to test the model against
  units=hp.Int('units', min_value=8, max_value=128, step=8)

  # defining layers
  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  # compiling model
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [25]:
# defining a tuner object
tuner= kt.RandomSearch(model_builder,
                       objective='val_accuracy',
                       max_trials=5,
                       directory='mydir',
                       project_name='keras-tuner-trial')

In [26]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 10s


In [27]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [28]:
best_model=tuner.get_best_models()[0]

In [29]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               1080      
                                                                 
 dense_1 (Dense)             (None, 1)                 121       
                                                                 
Total params: 1201 (4.69 KB)
Trainable params: 1201 (4.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
best_model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5714 - accuracy: 0.7182 - val_loss: 0.5312 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5302 - accuracy: 0.7476 - val_loss: 0.5006 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5051 - accuracy: 0.7524 - val_loss: 0.4838 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4869 - accuracy: 0.7655 - val_loss: 0.4719 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4759 - accuracy: 0.7622 - val_loss: 0.4644 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4697 - accuracy: 0.7622 - val_loss: 0.4615 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4621 - accuracy: 0.7720 - val_loss: 0.4627 - val_accuracy: 0.7922
Epoch 14

### **Selecting number of layers**

In [31]:
def model_builder(hp):
  # initializing sequential object
  model = Sequential()

  # setting the input layer
  model.add(Dense(48, activation='relu', input_dim=8))

  # initializing loop to determine optimum number of layers
  for i in range(hp.Int('layer_num', min_value=1, max_value=10)):

    model.add(Dense(48, activation='relu'))

  # setting up the output layer
  model.add(Dense(1, activation='sigmoid'))

  # compiling the model
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [32]:
# initializing a tuner
tuner=kt.RandomSearch(model_builder,
                      objective='val_accuracy',
                      max_trials=5,
                      directory='mydir',
                      project_name='number-of-layers')

In [33]:
# searching for the best hyperparameter
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 15s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'layer_num': 1}

In [35]:
best_model=tuner.get_best_models(num_models=1)[0]

In [36]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                432       
                                                                 
 dense_1 (Dense)             (None, 48)                2352      
                                                                 
 dense_2 (Dense)             (None, 1)                 49        
                                                                 
Total params: 2833 (11.07 KB)
Trainable params: 2833 (11.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
best_model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4640 - accuracy: 0.7655 - val_loss: 0.4875 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4597 - accuracy: 0.7687 - val_loss: 0.4855 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4518 - accuracy: 0.7638 - val_loss: 0.4855 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4479 - accuracy: 0.7736 - val_loss: 0.4832 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4436 - accuracy: 0.7736 - val_loss: 0.4840 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4420 - accuracy: 0.7850 - val_loss: 0.4857 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4381 - accuracy: 0.7850 - val_loss: 0.4832 - val_accuracy: 0.7987
Epoch 14

### **Tuning all parameters at once, where each layer has the same set of parameters.**

In [38]:
def model_builder(hp):

  model= Sequential()

  counter=0

  units=hp.Int('units', min_value=8, max_value=128, step=8)
  activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:
      model.add(
          Dense(
              units=units,
              activation=activation,
              input_dim=8
              )
          )
      counter+=1
    else:
      model.add(
          Dense(
              units=units,
              activation=activation
              )
          )
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop','sgd','adadelta','nadam']),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model



In [39]:
tuner=kt.RandomSearch(model_builder,
                      objective='val_accuracy',
                      max_trials=3,
                      directory='mydir',
                      project_name='all-in-one-tuner1'
                      )

In [40]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 11s


In [41]:
tuner.get_best_hyperparameters()[0].values

{'units': 72, 'activation': 'tanh', 'num_layers': 7, 'optimizer': 'sgd'}

In [42]:
best_model=tuner.get_best_models()[0]

In [43]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dense_1 (Dense)             (None, 72)                5256      
                                                                 
 dense_2 (Dense)             (None, 72)                5256      
                                                                 
 dense_3 (Dense)             (None, 72)                5256      
                                                                 
 dense_4 (Dense)             (None, 72)                5256      
                                                                 
 dense_5 (Dense)             (None, 72)                5256      
                                                                 
 dense_6 (Dense)             (None, 72)                5

In [44]:
best_model.fit(x_train, y_train, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 15ms/step - loss: 0.4877 - accuracy: 0.7573 - val_loss: 0.4737 - val_accuracy: 0.7987
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4800 - accuracy: 0.7687 - val_loss: 0.4674 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4759 - accuracy: 0.7704 - val_loss: 0.4682 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4735 - accuracy: 0.7752 - val_loss: 0.4663 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4706 - accuracy: 0.7769 - val_loss: 0.4652 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4690 - accuracy: 0.7687 - val_loss: 0.4645 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4684 - accuracy: 0.7769 - val_loss: 0.4674 - val_accuracy: 0.7857
Epoch 14

### **Tuning all parameters at once, where each layer has its own set of parameters.**

In [45]:
def model_builder(hp):

  model= Sequential()

  counter=0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:
      model.add(
          Dense(
              units=hp.Int('units-'+str(i), min_value=8, max_value=128, step=8),
              activation=hp.Choice('activation-'+str(i), values=['relu', 'tanh', 'sigmoid']),
              input_dim=8
              )
          )
      counter+=1
    else:
      model.add(
          Dense(
              units=hp.Int('units-'+str(i), min_value=8, max_value=128, step=8),
              activation=hp.Choice('activation-'+str(i), values=['relu', 'tanh', 'sigmoid'])
              )
          )
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop','sgd','adadelta','nadam']),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [46]:
tuner=kt.RandomSearch(model_builder,
                      objective='val_accuracy',
                      max_trials=5,
                      directory='mydir',
                      project_name='all-in-one-tuner-2'
                      )

In [47]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 14s


In [48]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7,
 'units-0': 120,
 'activation-0': 'tanh',
 'optimizer': 'adam',
 'units-1': 24,
 'activation-1': 'tanh',
 'units-2': 56,
 'activation-2': 'relu',
 'units-3': 8,
 'activation-3': 'tanh',
 'units-4': 104,
 'activation-4': 'relu',
 'units-5': 120,
 'activation-5': 'relu',
 'units-6': 8,
 'activation-6': 'sigmoid',
 'units-7': 80,
 'activation-7': 'relu',
 'units-8': 72,
 'activation-8': 'relu'}

In [49]:
best_model=tuner.get_best_models()[0]

In [50]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               1080      
                                                                 
 dense_1 (Dense)             (None, 24)                2904      
                                                                 
 dense_2 (Dense)             (None, 56)                1400      
                                                                 
 dense_3 (Dense)             (None, 8)                 456       
                                                                 
 dense_4 (Dense)             (None, 104)               936       
                                                                 
 dense_5 (Dense)             (None, 120)               12600     
                                                                 
 dense_6 (Dense)             (None, 8)                 9

In [51]:
best_model.fit(x_train, y_train, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 [==============================] - 2s 21ms/step - loss: 0.5074 - accuracy: 0.7410 - val_loss: 0.4845 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4925 - accuracy: 0.7508 - val_loss: 0.4772 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4844 - accuracy: 0.7687 - val_loss: 0.4753 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4806 - accuracy: 0.7720 - val_loss: 0.4821 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4724 - accuracy: 0.7834 - val_loss: 0.4787 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4699 - accuracy: 0.7785 - val_loss: 0.4793 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4639 - accuracy: 0.7818 - val_loss: 0.4787 - val_accuracy: 0.7727
Epoch 14

### **All in one with dropout**

In [53]:
def model_builder(hp):

  model= Sequential()

  counter=0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter==0:
      model.add(
          Dense(
              units=hp.Int('units-'+str(i), min_value=8, max_value=128, step=8),
              activation=hp.Choice('activation-'+str(i), values=['relu', 'tanh', 'sigmoid']),
              input_dim=8
              )
          )
      model.add(Dropout(hp.Choice('dropout-'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      counter+=1
    else:
      model.add(
          Dense(
              units=hp.Int('units-'+str(i), min_value=8, max_value=128, step=8),
              activation=hp.Choice('activation-'+str(i), values=['relu', 'tanh', 'sigmoid'])
              )
          )
      model.add(Dropout(hp.Choice('dropout-'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop','sgd','adadelta','nadam']),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [55]:
tuner= kt.RandomSearch(model_builder,
                       objective='val_accuracy',
                       max_trials=10,
                       directory='mydir',
                       project_name='tuner-with-dropout')

In [56]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 34s


In [57]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units-0': 128,
 'activation-0': 'sigmoid',
 'dropout-0': 0.7,
 'optimizer': 'adam',
 'units-1': 40,
 'activation-1': 'relu',
 'dropout-1': 0.4,
 'units-2': 112,
 'activation-2': 'sigmoid',
 'dropout-2': 0.2,
 'units-3': 32,
 'activation-3': 'sigmoid',
 'dropout-3': 0.6,
 'units-4': 72,
 'activation-4': 'sigmoid',
 'dropout-4': 0.5,
 'units-5': 48,
 'activation-5': 'sigmoid',
 'dropout-5': 0.2,
 'units-6': 120,
 'activation-6': 'relu',
 'dropout-6': 0.3,
 'units-7': 96,
 'activation-7': 'relu',
 'dropout-7': 0.6,
 'units-8': 112,
 'activation-8': 'relu',
 'dropout-8': 0.6,
 'units-9': 96,
 'activation-9': 'sigmoid',
 'dropout-9': 0.2}

In [58]:
best_model=tuner.get_best_models()[0]

In [59]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1152      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1281 (5.00 KB)
Trainable params: 1281 (5.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
best_model.fit(x_train, y_train, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.7337 - accuracy: 0.5814 - val_loss: 0.5921 - val_accuracy: 0.6623
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7166 - accuracy: 0.6156 - val_loss: 0.5777 - val_accuracy: 0.6623
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6683 - accuracy: 0.6336 - val_loss: 0.5568 - val_accuracy: 0.6948
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6485 - accuracy: 0.6336 - val_loss: 0.5458 - val_accuracy: 0.7273
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6434 - accuracy: 0.6596 - val_loss: 0.5376 - val_accuracy: 0.7273
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6750 - accuracy: 0.6612 - val_loss: 0.5292 - val_accuracy: 0.7208
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6489 - accuracy: 0.6694 - val_loss: 0.5216 - val_accuracy: 0.7468
Epoch 14

In summary, while a higher validation accuracy compared to the training accuracy typically signals overfitting, the presence of dropout regularization, especially with a high dropout rate like 0.7, can lead to disparities between the two accuracies. Dropout introduces randomness during training, causing fluctuations in performance and potentially resulting in a lower training accuracy. Therefore, in the context of dropout, differences in accuracies between training and validation sets may not always indicate overfitting but rather reflect the impact of dropout regularization on the model's training dynamics.